<h1><b>Machine Learning Based Clustering and Segmentation for Navigation<b></h1>

<h3><b>Introduction</b></h3>
    <p>
    An ML based navigation algorithm that is based on several factors pertaining to neighbourhoods. That will give you the most efficient route to the desired destination, based on factors such as crime rate and population density.
    </p>
<h3><b>Prerequisites</b></h3>
<ul>
    <li>Foursquare API</li>
</ul>
<h3><b>Datasets Used</b></h3>

<h3><b>Import Statements</b></h3>

In [158]:
from dotenv import load_dotenv
from dotenv import dotenv_values
import folium
import requests
import pandas as pd 
from pandas import json_normalize

<h3><b>Foursquare API Initialization / Check</b></h3>
<h4><b>Category Codes:</b></h4>
<ul>
    <li>10000 - Arts and Entertainment</li>
    <li>11000 - Business and Professional Services</li>
    <li>12000 - Community and Government</li>
    <li>13000 - Dining and Drinking</li>
    <li>14000 - Event</li>
    <li>15000 - Health and Medicine</li>
    <li>16000 - Landmarks and Outdoors</li>
    <li>17000 - Retail</li>
    <li>18000 - Sports and Recreation</li>
    <li>19000 - Travel and Transportation</li>
</ul>

In [262]:
config = dotenv_values(".env")
url = "https://api.foursquare.com/v3/places/nearby"

headers = {"Accept": "application/json",
            "Authorization": config["API_KEY"]}

response = requests.request("GET", url, headers=headers)

def create_request(coords= None, location = None, categories = None, limit = "10"):
    """
        Important:
            - Coords and location cannot be entered together
            - Location and radius cannot be entered together

        The coords will be a list with latitude and longitude.\n 
        Location will be a city and province such as  "Oshawa, ON".\n
        The category is a string from the above codes, with a default of None.\n
        The limit parameter is a maximum of 50, with a default of 10 requests.\n

        Examples:
            - create_request(coords=[-72.848752,43.895962], limit="1")
            - create_request(coords=[-72.848752,43.895962], categories="10000", limit="2")\n
            - create_request(location=["Oshawa","ON"], limit="2")
            - create_request(location=["Oshawa","ON"], categories="10000", limit="20")
    """

    if(coords and categories == None):
        url = "https://api.foursquare.com/v3/places/search?ll=" + str(coords[0]) + "%2C" + str(coords[1]) + "&radius=100000"  + "&limit=" + limit
    elif(coords and categories):
        url = "https://api.foursquare.com/v3/places/search?ll=" + str(coords[0]) + "%2C" + str(coords[1]) +"&categories=" + categories + "&radius=100000" + "&limit=" + limit
    elif(location and categories == None):
        url = "https://api.foursquare.com/v3/places/search?" + "near=" + str(location[0]) + "%2C" + str(location[1]) + "&limit=" + limit
    elif(location and categories):
        url = "https://api.foursquare.com/v3/places/search?" + "categories=" + categories + "&near=" + str(location[0]) + "%2C" + str(location[1]) + "&limit=" + limit
    else:
        return False
    
    response = requests.request("GET", url, headers=headers)
    
    if(response.status_code == 200):
        return response.json()
    else:
        return False

<h3><b>Creating DataFrames</b></h3>

In [290]:
latitude = 43.6532 
longitude = -79.3832
results = create_request(location = ["Toronto", "ON"], categories="13000", limit="10")

# Generates data from the Foursquare API
venues = json_normalize(results['results'], max_level=3)
venues.drop(venues.columns[[0, 1, 2, 3, 5, 8, 9, 10, 12, 17, 18, 19, 20, 21]], axis=1, inplace=True)

#
pd.DataFrame(venues)





,name,geocodes.main.latitude,geocodes.main.longitude,location.country,location.formatted_address,location.locality,location.neighborhood,location.postcode
0,Contra Cafe,43.668930,-79.426097,CA,"1028 Shaw St (Yarmouth), Toronto ON M6G 3N1",Toronto,[Downtown Toronto],M6G 3N1
1,Blood Brothers Brewing,43.669919,-79.436405,CA,"165 Geary Ave (at Barlett Ave N), Toronto ON M...",Toronto,[West Toronto],M6H 2B8
2,The Keg Steakhouse + Bar,43.705685,-79.397912,CA,"2201 Yonge St (at Eglinton Ave), Toronto ON M4...",Toronto,[Davisville],M4S 2B2
3,Burdock,43.659485,-79.437818,CA,"1184 Bloor St W, Toronto ON M6H 1N2",Toronto,[Wallace Emerson],M6H 1N2
4,The Library Specialty Coffee,43.654250,-79.390973,CA,"281 Dundas St W (St Patrick And Dundas St W), ...",Toronto,[Downtown Toronto],M5T 1G1
5,Rustic Bakery,43.715504,-79.490353,CA,"318 Rustic Rd (at Culford Rd.), North York ON ...",North York,[York],M6L 1W5
6,Cheese Boutique,43.638456,-79.475109,CA,"45 Ripley Ave (South Kingsway), Toronto ON M6S...",Toronto,[Swansea],M6S 3P2
7,Photien Thanh Vietnamese Restaurant,43.645527,-79.419079,CA,"57 Ossington Ave (at Queen St. W.), Toronto ON...",Toronto,[Trinity - Bellwoods],M6J 2Y9
8,Eataly,43.669563,-79.388831,CA,"55 Bloor St W (at Bay St), Toronto ON M4W 1A5",Toronto,[Downtown Toronto],M4W 1A5
9,Hair of the Dog,43.662997,-79.379508,CA,"425 Church St (at Wood St.), Toronto ON M4Y 2C3",Toronto,[Downtown Toronto],M4Y 2C3


<h3><b>Map Creation</b></h3>


In [296]:
map_creation = folium.Map(location=[latitude, longitude], zoom_start=10)
map_creation